In [ ]:
import glob
import pandas as pd
import uproot
import uproot_methods
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
import matplotlib.colors as mcolors

%matplotlib inline
import mplhep as hep
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from mpl_toolkits.axes_grid1 import make_axes_locatable


plt.style.use(hep.style.ROOT) # For now ROOT defaults to CMS

import numpy as np
from scipy.stats import norm
from scipy.optimize import curve_fit
from scipy import interpolate

import itertools

import functions_giammi as fugi
cmap_temp, cmap_temp_go, Magma_r, Seismic_r = fugi.customcmaps()

dir = r"../PYTHON_graphs/OUTPUTS/N2_calibration/"
allfiles = (glob.glob("../PYTHON_graphs/DATA/Standard_grpahs/*.root"))

cmap_viridis = mpl.cm.get_cmap('viridis')

In [ ]:
allfiles

In [ ]:
elec_lin = uproot.open(allfiles[0])
pipico_n2 = uproot.open(allfiles[1])
recoil_sum_w = uproot.open(allfiles[2])
recoil_xy = uproot.open(allfiles[3])
pipico_n2_momcon = uproot.open(allfiles[4])
elec_sum_w = uproot.open(allfiles[5])
elec_xy = uproot.open(allfiles[6])

In [ ]:
#N2
w_layer,tsum,counts_el_sum=fugi.import_TH2Dgeneric(elec_sum_w,elec_sum_w.keys()[0])
x_el,y_el,counts_el_xy=fugi.import_TH2Dgeneric(elec_xy,elec_xy.keys()[0])
x_rec,y_rec,counts_rec_xy=fugi.import_TH2Dgeneric(recoil_xy,recoil_xy.keys()[0])
x_el_lin,y_el_lin,counts_el_lin=fugi.import_TH2Dgeneric(elec_lin,elec_lin.keys()[0])
tof1,tof2,counts_pipico=fugi.import_TH2Dgeneric(pipico_n2_momcon,pipico_n2_momcon.keys()[0])

In [ ]:
#sum el w-layer
fig,ax= plt.subplots(1, figsize=(12, 10))
counts_el_sum_red=np.ma.masked_array(counts_el_sum,counts_el_sum<3)

# cs=ax.contourf(w_layer,tsum,counts_el_sum_red.T, levels=40)
cs=ax.pcolormesh(w_layer,tsum,counts_el_sum_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('w-layer [ns]')
ax.set_ylabel('time sum [ns]')
ax.set_ylim(-1,1)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_el_sum_red.min(),counts_el_sum_red.max()])
cbar.set_ticklabels(["min","max"])
cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"el_sum_wlayer.png", dpi=300, transparent=False)
# fig.savefig(dir+"el_sum_wlayer.svg", dpi=300, transparent=False)

In [ ]:
xy_el=np.array(list(itertools.product(x_el, y_el)))
X_el=np.array([col[0] for col in xy_el])
Y_el=np.array([col[1] for col in xy_el])

xy_rec=np.array(list(itertools.product(x_rec, y_rec)))
X_rec=np.array([col[0] for col in xy_rec])
Y_rec=np.array([col[1] for col in xy_rec])

In [ ]:
#3D MFPAD
counts_el_xy_new=np.ma.masked_where(np.sqrt(X_el**2+Y_el**2)<25,counts_el_xy.reshape(-1)).reshape(len(x_el),len(y_el))
counts_el_xy_new_red=np.ma.masked_array(counts_el_xy_new, counts_el_xy_new<=5)
norm=mcolors.LogNorm(counts_el_xy_new_red.min(),counts_el_xy_new_red.max())

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_el.max()/y_el.max())
# cs=ax.contourf(x_el,y_el,counts_el_xy_new_red.T, levels=20)
cs=ax.pcolormesh(x_el,y_el,counts_el_xy_new_red.T, shading='gouraud')
# cs=ax.pcolormesh(x_el,y_el,counts_el_xy_new_red.T, shading='gouraud', norm=norm)
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)
ax.plot([0, 20], [0, -33], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.plot([0, -33], [0, -19.5], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.plot([0, 19], [0, 32], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.grid()

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_el_xy_new_red.min(),counts_el_xy_new_red.max()])
cbar.set_ticklabels(["min","max"])

cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"MCP_el_xy_red.png", dpi=300, transparent=False)
# fig.savefig(dir+"MCP_el_xy_red.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD

counts_rec_xy_new=np.ma.masked_where(np.sqrt(X_rec**2+Y_rec**2)<25,counts_rec_xy.reshape(-1)).reshape(len(x_rec),len(y_rec))
counts_rec_xy_new_red=np.ma.masked_array(counts_rec_xy_new, counts_rec_xy_new<=5)
norm=mcolors.LogNorm(counts_rec_xy_new_red.min(),counts_rec_xy_new_red.max())

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_rec.max()/y_rec.max())
# cs=ax.contourf(x_rec,y_rec,counts_rec_xy_new_red.T, levels=20)
# cs=ax.pcolormesh(x_rec,y_rec,counts_rec_xy_new_red.T, shading='gouraud')
cs=ax.pcolormesh(x_rec,y_rec,counts_rec_xy_new_red.T, shading='gouraud', norm=norm)
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)
ax.plot([0, 19], [0, -33], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.plot([0, -33], [0, -19.5], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.plot([0, 19], [0, 32], color='black', linewidth=3., linestyle='--', alpha=0.5)
ax.grid()

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_rec_xy_new_red.min(),counts_rec_xy_new_red.max()])
cbar.set_ticklabels(["min","max"])

cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"MCP_rec_xy_red.png", dpi=300, transparent=False)
# fig.savefig(dir+"MCP_rec_xy_red.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
counts_el_xy_red=np.ma.masked_array(counts_el_xy,counts_el_xy<1)
norm=mcolors.LogNorm(counts_el_xy_red.min(),counts_el_xy_red.max())

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_el.max()/y_el.max())

# cs=ax.contourf(x_el,y_el,counts_el_xy_red.T, levels=20)
# cs=ax.pcolormesh(x_el,y_el,counts_el_xy_red.T, shading='gouraud', norm=norm)
cs=ax.pcolormesh(x_el,y_el,counts_el_xy_red.T, shading='gouraud')
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_el_xy_red.min(),counts_el_xy_red.max()])
cbar.set_ticklabels(["min","max"])

cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"el_mom_ex.png", dpi=300, transparent=False)
# fig.savefig(dir+"el_mom_ex.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
x_el_lin_r=fugi.remap(x_el_lin,-45,45)
y_el_lin_r=fugi.remap(y_el_lin,-45,45)
counts_el_lin_red=np.ma.masked_array(counts_el_lin,counts_el_lin<0.00001)
norm=mcolors.LogNorm(counts_el_lin_red.min(),counts_el_lin_red.max())

fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(x_el.max()/y_el.max())
# cs=ax.contourf(x_el_lin_r,y_el_lin_r,counts_el_lin_red, levels=40)
# cs=ax.pcolormesh(x_el_lin_r,y_el_lin_r,counts_el_lin_red, shading='gouraud')
cs=ax.pcolormesh(x_el_lin_r,y_el_lin_r,counts_el_lin_red.T, shading='gouraud', norm=norm)
# ax.set_title("fish fillet y-axis")

ax.set_xlabel('x-axis [mm]')
ax.set_ylabel('y-axis [mm]')
ax.set_aspect(aspect)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
# cbar.set_ticks([counts_el_lin_red.min(),counts_el_lin_red.max()])
# cbar.set_ticklabels(["min","max"])

cbar.ax.set_ylabel('deviation from linearity')

fig.savefig(dir+"MCP_el_linearity.png", dpi=300, transparent=False)
# fig.savefig(dir+"MCP_el_linearity.svg", dpi=300, transparent=False)

In [ ]:
#3D MFPAD
fig,ax= plt.subplots(1, figsize=(12, 10))
aspect=(tof1.max()/tof2.max())
counts_pipico_red=np.ma.masked_array(counts_pipico,counts_pipico<1)
# ax.set_title("fish fillet y-axis")

cs=ax.pcolormesh(tof1,tof2,counts_pipico_red.T, shading='gouraud')

ax.set_xlabel('tof1 [ns]')
ax.set_ylabel('tof2 [ns]')
ax.set_aspect(aspect)

axins = ax.inset_axes([0.55, 0.06, 0.42, 0.42])
axins.pcolormesh(tof1,tof2,np.ma.masked_array(counts_pipico.T,counts_pipico.T<1), shading='gouraud')
axins.set_xlim(2800,3100)
axins.set_ylim(2900,3200)
axins.tick_params(axis='x', labelsize=12)
axins.tick_params(axis='y', labelsize=12)

cbar = fig.colorbar(cs,shrink=0.9, ax=ax)
cbar.set_ticks([counts_pipico_red.min(),counts_pipico_red.max()])
cbar.set_ticklabels(["min","max"])

cbar.ax.set_ylabel('normalized counts')

fig.savefig(dir+"N2_pipico_insert.png", dpi=300, transparent=False)
# fig.savefig(dir+"N2_pipico_insert.svg", dpi=300, transparent=False)